In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


from imblearn.over_sampling import SMOTE, ADASYN


# from xgboost import XGBClassifier
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


In [19]:
df = pd.read_csv("/kaggle/input/personal-key-indicators-of-heart-disease/heart_2020_cleaned.csv")
df.head()


In [20]:
df.info()

In [21]:
df.isna().sum()

In [22]:
plt.figure(figsize=(7,7))
sns.countplot(data=df,x="HeartDisease",hue="Asthma");

In [23]:
plt.figure(figsize=(7,7))
sns.countplot(data=df,x="HeartDisease",hue="KidneyDisease");

In [24]:
plt.figure(figsize=(7,7))
sns.countplot(data=df,x="HeartDisease",hue="Stroke");

In [25]:
plt.figure(figsize=(7,7))
sns.countplot(data=df,x="HeartDisease",hue="SkinCancer");

In [27]:
plt.figure(figsize=(7,7))
sns.countplot(data=df,x="HeartDisease",hue="DiffWalking");

In [12]:
# plt.figure(figsize=(7,7))
# sns.countplot(data=df,x="HeartDisease",hue="MentalHealth")

In [28]:
df_2 = df.copy()
dict(df_2["Diabetic"].value_counts())

Yes_No_dict = {"No":0, "Yes":1, "Female":1, "Male":0} 
#                ,"No, borderline diabetes": 2, 
#                "Yes (during pregnancy)": 3}

Yes_No_List = ["Stroke", "HeartDisease", "Smoking", "AlcoholDrinking","PhysicalActivity", "Sex"]#, "Diabetic"]

for col in Yes_No_List:
    df_2[col] = df_2[col].map(Yes_No_dict)

df_2.head()

In [29]:
df_2["HeartDisease"].value_counts(normalize = True)

In [30]:
sns.histplot(df_2["BMI"])
plt.show()

In [31]:
df_2.columns

In [36]:


List_1_0 = ["Stroke", "Smoking", "AlcoholDrinking","PhysicalActivity", "Sex"]#, "Diabetic"]

# List_1_0 = ["Smoking", "AlcoholDrinking","PhysicalActivity", "Sex"] # No "Stroke", 

df_List_1_0 = df_2[List_1_0]

#### Encoding By using OneHotEncoder

cat = ["Diabetic", "Race", "GenHealth" ] #, "SleepTime"] 
# cat = ["Race", "GenHealth"] #, "SleepTime", "Diabetic" ]
df_cat = df_2[cat]



ohe = OneHotEncoder(categories='auto', sparse=False, handle_unknown = "ignore", drop="first")

df_cat_encoded = pd.DataFrame(np.squeeze(ohe.fit_transform(df_cat)), columns = ohe.get_feature_names())

##### Scaling By using StandardScaler

scaler = StandardScaler()

BMI = df_2[["BMI", "SleepTime"]]
# BMI = df_2[["BMI"]]

scaled_BMI = scaler.fit_transform(BMI)

df_scaled_BMI = pd.DataFrame(np.squeeze(scaler.fit_transform(BMI)), columns = scaler.feature_names_in_)

#### Concatating New DataFrames

X = pd.concat([df_cat_encoded, df_scaled_BMI, df_List_1_0], axis = 1)
# X = pd.concat([df_cat_encoded, BMI, df_List_1_0], axis = 1)

# X = pd.concat([BMI, df_List_1_0], axis = 1)

X.head()

In [37]:

y = df_2["HeartDisease"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify=y)

# smote = SMOTE()
# X_train, y_train = smote.fit_resample(X_train, y_train)

X_train_resampled = X_train.copy()
y_train_resampled = y_train.copy()

In [38]:
print("y_train:", dict(y_train_resampled.value_counts(normalize = True)*100))
print("y_test:", dict(y_test.value_counts(normalize = True)*100))

**XGBoost**

In [44]:
clf = xgb.XGBClassifier(max_depth = 20, n_estimators=20, seed=42, alpha = 25,  eta = 0.3,
                        objective = 'binary:logistic', eval_metric = "error")
clf.fit(X_train_resampled, y_train_resampled)

training_preds = clf.predict(X_train_resampled)
test_preds     = clf.predict(X_test)


training_accuracy =  accuracy_score(y_train_resampled, training_preds)
test_accuracy     =  accuracy_score(y_test, test_preds)

print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

# print(sum(test_preds == y_test)/len(y_test))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15, 7))
xgb.plot_importance(clf, ax = ax);

In [ ]:
#### 
# No "Stroke", and "SleepTime" not categorical and is in BMI
# max_depth = 10, n_estimators=10, seed=42, alpha = 20, eta = 0.3, objective = 'binary:logistic', 
# eval_metric = "error"```

# gives 

# Training Accuracy: 91.51%
# Validation accuracy: 91.51%
#---------------------------------------------------------------------------------------------
### However, with stroke and and "SleepTime" not categorical and is in BMI
# max_depth = 10, n_estimators=10, seed=42, alpha = 20, eta = 0.3,
#                         objective = 'binary:logistic', eval_metric = "error"

# We got

# Training Accuracy: 91.61%
# Validation accuracy: 91.58%

#---------------------------------------------------------------------------------------------
### However, with stroke and and "SleepTime" as categorical and not in BMI
# max_depth = 10, n_estimators=10, seed=42, alpha = 20, eta = 0.3,
#                         objective = 'binary:logistic', eval_metric = "error"

# We got

# Training Accuracy: 91.62%
# Validation accuracy: 91.59%

#---------------------------------------------------------------------------------------------

### With Diabetic as a Categorical with ranges from 0-3 in the loop, and "SleepTime" in BMI, 
### with  drop="first"
### and eval_metric = "error" and max_depth = 20, n_estimators=20, seed=42, alpha = 25,  eta = 0.3, 
### objective = 'binary:logistic', 
### eval_metric = "error"

### we got

### Training Accuracy: 91.62%
### Validation accuracy: 91.6%

#---------------------------------------------------------------------------------------------



**GridSearch For XGBoost**

In [60]:
clf_param_grid = {
    "max_depth": [10, 20, 25],
    "n_estimators": [15, 20, 25, 30],
    "seed": [42],
    "alpha": [20, 25, 30],
    "eta": [0.3],
    "objective" : ['binary:logistic'], 
    "eval_metric" : ["auc", "error"]
}

In [61]:
clf_grid = xgb.XGBClassifier()

clf_grid_search = GridSearchCV(clf_grid, clf_param_grid, cv = 2)


# Fit to the data
clf_grid_search.fit(X_train_resampled, y_train_resampled)

In [62]:
grid_training_preds = clf_grid_search.predict(X_train_resampled)
grid_test_preds     = clf_grid_search.predict(X_test)


grid_training_accuracy =  accuracy_score(y_train_resampled, grid_training_preds)
grid_test_accuracy     =  accuracy_score(y_test, grid_test_preds)

print('Training Accuracy: {:.4}%'.format(grid_training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(grid_test_accuracy * 100))

In [63]:
clf_grid_search.best_params_

**Random Forest**

In [64]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion="entropy", bootstrap = True)

rf.fit(X_train_resampled, y_train_resampled)

training_preds = rf.predict(X_train_resampled)
test_preds     = rf.predict(X_test)


training_accuracy = accuracy_score(y_train_resampled, training_preds)
test_accuracy     = accuracy_score(y_test, test_preds)

print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

In [ ]:
# param_grid ={"max_depth": [5, 10, 15, 20],
#             "min_samples_leaf": [5, 10, 15],
#             "min_samples_split": [2, 3, 6]}

In [ ]:

# grid_clf = GridSearchCV(rf, param_grid, scoring='accuracy', cv=None,
#                                                              n_jobs=1)
# grid_clf.fit(X_train_resampled, y_train_resampled)

# best_parameters = grid_clf.best_params_

# print('Grid Search found the following optimal parameters: ')
# for param_name in sorted(best_parameters.keys()):
#     print('%s: %r' % (param_name, best_parameters[param_name]))

# training_preds = grid_clf.predict(X_train_resampled)
# test_preds     = grid_clf.predict(X_test)
# training_accuracy = accuracy_score(y_train_resampled, training_preds)
# test_accuracy = accuracy_score(y_test, test_preds)

# print('')
# print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
# print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))